In [1]:
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
import numpy as np
import os
from glob import glob
import os
from PIL import Image
import time
import numpy as np
import matplotlib.pyplot as plt
import scipy
import keras.backend as K

import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
img_rows = 512
img_cols = 512
channels = 3
img_shape = (img_rows, img_cols, channels)
img_res=(512, 512)
dataset_name = 'PIX'
img_res = img_res
image_size = 512
img_rows = 512
img_cols = 512
channels = 3
img_shape = (img_rows, img_cols, channels)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)
disc_loss =[]
gen_loss = []
# Number of filters in the first layer of G and D
gf = 128
df = 128
optimizer = Adam(0.0001, 0.5)
gf = 128
df = 128

optimizer = Adam(0.0002, 0.5)
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)



In [3]:
def load_data( batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "test"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        if batch_size > 0:
            batch_images = np.random.choice(path, size=batch_size)
        else:
            batch_images = np.array([])
        #batch_images = np.random.choice(path, size=batch_size)

        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = imread(img_path)

            h, w, _ = img.shape
            _w = int(w/2)
            img_A, img_B = img[:, :_w, :], img[:, _w:, :]

            img_A = scipy.misc.imresize(img_A, img_res)
            img_B = scipy.misc.imresize(img_B, img_res)

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

In [4]:
def load_batch(batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        n_batches = int(len(path) / batch_size)

        for i in range(n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, img_res)
                img_B = scipy.misc.imresize(img_B, img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [5]:
def m_batch(batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (dataset_name, data_type))
        batch_size=3
        m_batches = int(len(path) / batch_size)
        return m_batches

In [6]:
def imread(path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)
    

In [7]:
def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

In [8]:
def conv2d(layer_input, filters, f_size=4, bn=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

In [9]:
 def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

In [10]:
def build_generator():
        """U-Net Generator"""

        

        # Image input
        d0 = Input(shape=img_shape)

        # Downsampling
        d1 = conv2d(d0, gf, bn=False)
        d2 = conv2d(d1, gf*2)
        d3 = conv2d(d2, gf*4)
        d4 = conv2d(d3, gf*8)
        d5 = conv2d(d4, gf*8)
        d6 = conv2d(d5, gf*8)
        d7 = conv2d(d6, gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, gf*8)
        u2 = deconv2d(u1, d5, gf*8)
        u3 = deconv2d(u2, d4, gf*8)
        u4 = deconv2d(u3, d3, gf*4)
        u5 = deconv2d(u4, d2, gf*2)
        u6 = deconv2d(u5, d1, gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

In [11]:
def build_discriminator():

        

        img = Input(shape=img_shape)

        d1 = d_layer(img, df, normalization=False)
        d2 = d_layer(d1, df*2)
        d3 = d_layer(d2, df*4)
        d4 = d_layer(d3, df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

In [12]:
def wasserstein_loss( y_true, y_pred):
    return K.mean(y_true * y_pred)

In [13]:
D_A = build_discriminator()
D_A.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
D_B = build_discriminator()
D_B.compile(loss=wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
G_AB = build_generator()
G_BA = build_generator()

        # For the combined model we will only train the generators
D_A.trainable = False
D_B.trainable = False

        
        # Generators translates the images to the opposite domain
fake_B = G_AB(img_A)
fake_A = G_BA(img_B)

# The discriminators determines validity of translated images
valid_A = D_A(fake_A)
valid_B = D_B(fake_B)

# Generators translate the images back to their original domain
recov_A = G_BA(fake_B)
recov_B = G_AB(fake_A)

        # The combined model  (stacked generators and discriminators)
combined = Model(inputs=[img_A, img_B], outputs=[valid_A, valid_B, recov_A, recov_B])
combined.compile(loss=[wasserstein_loss,wasserstein_loss, 'mae', 'mae'],
                            optimizer=optimizer,
                            loss_weights=[1, 1, 100, 100])


In [14]:
def sample_images( epoch, batch_i):
        os.makedirs('images/%s' % dataset_name, exist_ok=True)
        r, c = 3, 3
        batch_size=2
        imgs_A, imgs_B = load_data(batch_size=2, is_testing=True)

        # Translate images to the other domain
        fake_B = g_AB.predict(imgs_A)
        fake_A = g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = g_BA.predict(fake_B)
        reconstr_B = g_AB.predict(fake_A)
        #display_imageGA(fake_B)
        #display_imageOA(imgs_A)
        #display_imageGB(fake_A)
        #display_imageOB(imgs_B)
        #display_imageRA(reconstr_A)
        #display_imageRB(reconstr_B)
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[j,i].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images12/DUAL/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
plt.close()

In [15]:
def train( epochs, batch_size=128, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + disc_patch)
        fake = np.zeros((batch_size,) + disc_patch)

        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = g_AB.predict(imgs_A)
                fake_A = g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))
                d_A.save_weights('./DISCO/weights/d_a_weights222.h5', True)
                d_B.save_weights('./DISCO/weights/d_b_weights222.h5', True)
                g_AB.save_weights('./DISCO/weights/G_a_weights222.h5', True)
                g_BA.save_weights('./DISCO/weights/G_b_weights222.h5', True)

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    sample_images(epoch, batch_i)
                    